In [4]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import numpy as np
import csv
import matplotlib.pyplot as plt
from astropy.io.votable import parse_single_table
import time

# ADQL used for GOODS South Star Chart, retrieves all objects within 100 arcmin:

# SELECT *, DISTANCE(
#    POINT(53.1, -27.8),
#    POINT(ra, dec))
# FROM gaiadr3.gaia_source
# WHERE 1 = CONTAINS(
#    POINT(53.1, -27.8),
#    CIRCLE(ra, dec, 100./60.))
# AND phot_g_mean_mag IS NOT NULL
# AND parallax IS NOT NULL
# ORDER BY phot_g_mean_mag ASC

In [71]:
#=====================================================================================================================
class FOV_stars: #Class for the FOV simulator
    def __init__(self,scienceFOV_radius=7.4/2,techFOV_radius=5,mag_limit=19,NSIDE=2**18):
        self.scienceFOV_radius=scienceFOV_radius #Diameter of the science FOV in arcmin
        self.techFOV_radius=techFOV_radius #Diameter of the tech FOV in arcmin
        self.mag_limit=mag_limit #Minimum magnitude of stars to analyse
        self.NSIDE=NSIDE # NSIDE correlates to the number of pixels in the HEALpix array, typically in powers of 2
        # NSIDE therefore determines resolution of the pixels, NSIDE=2**18 gives resolution of ~1 arcsec
        
        #Dummy LGS variables until requirements are provided:
        self.LGS_distance=3.5 #Distance of LGS cutouts to FOV centre in arcmin
        self.LGS_radius=1.5 #Radius of LGS cutouts in armic
#=====================================================================================================================
    def loadstars(self): #Loads the stars near GOODS South and coverts their positions to HEAL pixels
        #Loads GOODS SOUTH STAR CHART within 100 arcmin; downloaded using ASQL from GAIA DR3
        table = parse_single_table("CANDLES2")
        CANDLES=table.array

        self.ra=[] #Right Ascension
        self.dec=[] #Declination
        self.mag=[] #Magnitude
        for i in CANDLES[:]:    #Converts all mag, ra, and dec into floats from strings
            if i[2] != '':  #Some observations do not have a magnitude; so these are ignored
                if float(i[69]) < self.mag_limit: #Faint stars of mag {INSERT} and above are ignored (too faint for AO NGS)
                    self.mag.append(float(i[69])) 
                    self.ra.append(float(i[5])) 
                    self.dec.append(float(i[7]))

        start=time.time() #Timer to measure the time it takes to convert all stars to HEALpixels
        
        self.star_pixels=[] #All stars in the catalogue need to be converted to pixel numbers on the HEALpix array
        for i in range(0,len(self.ra)):
            self.star_pixels.append(hp.ang2pix(self.NSIDE,self.ra[i],self.dec[i],lonlat=True))
        
        period = time.time() - start
        print("Time to convert stars to pixels = {:.2}s".format(period))
#=====================================================================================================================
#Following function takes the pointing of the FOV and the angle of the LGS cutouts and returns the (ra,dec) of the LGS cutouts 
#Treats the cutout as co-ords on a unit sphere undergoing 2 rotations:
#1) rotation around the x-axis centred on the FOV to provide the orientation of the cutouts
#2) rotation around the y-axis perpendicular to the FOV to provide the correct declination of the cutouts

    def LGSradec(self,ra0,dec0,LGS_angle): 
        
        #Polar (theta,phi) co-ords of the LGS cutouts at a FOV of ra,dec=0,0
        LGS_polars=[[90-self.LGS_distance/60,0],[90,self.LGS_distance/60],[90+self.LGS_distance/60,0],[90,-self.LGS_distance/60]]
    
        LGS_angle_rad = np.radians(-LGS_angle) #LGS rotation angle needs to be in radians (-symbol as rotation matrix is clockwise from +x)
        dec0_rad=np.radians(dec0) #dec0 needs to be in radians

        radec=[] #ra and dec co-ordinates of the LGS cutouts

        for i in range(0,4): #Goes through each of the 4 LGS cutouts
            theta=np.radians(LGS_polars[i][0])
            phi=np.radians(LGS_polars[i][1])
            
            #Following is conversion of polar (theta,phi) to (x,y,z)
            x = np.sin(theta)*np.cos(phi)
            y = np.sin(theta)*np.sin(phi)
            z = np.cos(theta)

            #Rotation around the "x-axis", centred on the FOV
            x_rotx = x
            y_rotx = y*np.cos(LGS_angle_rad)-z*np.sin(LGS_angle_rad)
            z_rotx = y*np.sin(LGS_angle_rad)+z*np.cos(LGS_angle_rad)

            #Rotation around the "y-axis", perpendicular to the FOV
            x_rotxy = x_rotx*np.cos(-dec0_rad)+z_rotx*np.sin(-dec0_rad)
            y_rotxy = y_rotx
            z_rotxy = x_rotx*-np.sin(-dec0_rad)+z_rotx*np.cos(-dec0_rad) 

            #New (theta,phi) co-ords for the LGS cutouts
            thetanew = np.degrees(np.arccos(z_rotxy))
            phinew = np.degrees(np.arctan(y_rotxy/x_rotxy))

            radec.append([phinew+ra0,90-thetanew]) 
            #+ra0 as need to re-align the right ascension, as we presumed ra=0 for the x-axis rotation
            #90- due to conversion of polar to lon/lat
            
        return radec
#=====================================================================================================================        
    def findstars(self,ra0,dec0,LGS_angle=0,report=False): #Finds stars within the science and tech FOVS
        #ra0 and dec0 are pointing of the telescope/centre of FOV
        #LGS angle is the angle to rotate the LGS footprint by, anticlockwise from north in degrees
        #Report set to true to returns processing time and resolution
        
        start = time.time() #Initial time of findstars, to calculate the sim's duration
        
        vec = hp.ang2vec(ra0,dec0,lonlat=True) #Converts pointing to HEALpix vector
        
        #Retrieves pixel numbers that are inside the science and tech FOVs
        scienceFOV_pixels = hp.query_disc(nside=self.NSIDE, vec=vec, radius=np.radians(self.scienceFOV_radius/60)) 
        techFOV_pixels = hp.query_disc(nside=self.NSIDE, vec=vec, radius=np.radians(self.techFOV_radius/60)) 
   
        scienceFOV_stars=[] #Indexes of stars within science FOV 
        techFOV_stars=[] #Indexes of stars within technical FOV 
        techFOVonly_stars=[] #Indexes of stars within technical FOV (aka annulus)

        #Following identifies which stars are inside the science and tech FOVs
        #Does thi by comparing the pixel numbers of individual stars with those within the FOV
        count=-1
        techFOV_index=[]
        scienceFOV_index=[]
        techFOVonly_index=[]
        for i in self.star_pixels:
            count=count+1
            if i in techFOV_pixels:
                techFOV_stars.append(i)
                techFOV_index.append(count)
                if i in scienceFOV_pixels:
                    scienceFOV_stars.append(i)
                    scienceFOV_index.append(count)
        count=-1

        for i in techFOV_stars:
            count=count+1

            if i not in scienceFOV_stars:
                techFOVonly_stars.append(i)
                techFOVonly_index.append(techFOV_index[count])
        
        #Calculates ra and dec of the  4 LGS cutouts based on the pointing and the angle the cutouts are rotated by relative to north
        LGS_radecs=self.LGSradec(ra0,dec0,LGS_angle)
        
        #Converts ra and dec of the LGS cutouts to HEALpix vectors
        LGS1_vec=hp.ang2vec(LGS_radecs[0][0],LGS_radecs[0][1],lonlat=True)
        LGS2_vec=hp.ang2vec(LGS_radecs[1][0],LGS_radecs[1][1],lonlat=True)
        LGS3_vec=hp.ang2vec(LGS_radecs[2][0],LGS_radecs[2][1],lonlat=True)
        LGS4_vec=hp.ang2vec(LGS_radecs[3][0],LGS_radecs[3][1],lonlat=True)

        #Retrieves pixel numbers that are inside each LGS cutout
        LGS1_pixels = hp.query_disc(nside=self.NSIDE, vec=LGS1_vec, radius=np.radians(self.LGS_radius/60)) 
        LGS2_pixels = hp.query_disc(nside=self.NSIDE, vec=LGS2_vec, radius=np.radians(self.LGS_radius/60))
        LGS3_pixels = hp.query_disc(nside=self.NSIDE, vec=LGS3_vec, radius=np.radians(self.LGS_radius/60)) 
        LGS4_pixels = hp.query_disc(nside=self.NSIDE, vec=LGS4_vec, radius=np.radians(self.LGS_radius/60))
        
        cutoutstars_index=[] #Indexes of stars within LGS cutouts
        cutoutstars_LGS=[] #Which LGS cutout each star lies in: {1,2,3,4}
        
        #Following identifies which FOV stars are inside specific LGS cutouts by comparing pixel numbers
        #Stars that are in LGS cutouts are removed from the scienceFOV and techFOV lists and are added to the cutout list
        count=-1
        footprint_index=[]
        print("tech")
        dupetechFOVonly_stars=techFOVonly_stars.copy()
        for i in dupetechFOVonly_stars:
            count=count+1
            if i in LGS1_pixels:
                cutoutstars_index.append(i)
                techFOVonly_stars.remove(i)
                cutoutstars_LGS.append(1)
                footprint_index.append(techFOVonly_index[count])
                print(self.mag[techFOVonly_index[count]])
                print("1")
            if i in LGS2_pixels:
                cutoutstars_index.append(i)
                techFOVonly_stars.remove(i)
                cutoutstars_LGS.append(2)
                footprint_index.append(techFOVonly_index[count])
                print(self.mag[techFOVonly_index[count]])
                print("2")
            if i in LGS3_pixels:
                cutoutstars_index.append(i)
                techFOVonly_stars.remove(i)
                cutoutstars_LGS.append(3)
                footprint_index.append(techFOVonly_index[count])
                print(self.mag[techFOVonly_index[count]])
                print("3")
            if i in LGS4_pixels:
                cutoutstars_index.append(i)
                techFOVonly_stars.remove(i)
                cutoutstars_LGS.append(4)  
                footprint_index.append(techFOVonly_index[count])
                print(self.mag[techFOVonly_index[count]])
                print("4")
        count=-1
        print("science")
        dupescienceFOV_stars=scienceFOV_stars.copy()
        for i in dupescienceFOV_stars:
            count=count+1
            if i in LGS1_pixels:
                cutoutstars_index.append(i)
                scienceFOV_stars.remove(i)
                cutoutstars_LGS.append(1)
                footprint_index.append(scienceFOV_index[count])
                print(self.mag[scienceFOV_index[count]])
                print("1")
            if i in LGS2_pixels:
                cutoutstars_index.append(i)
                scienceFOV_stars.remove(i)
                cutoutstars_LGS.append(2)  
                print(self.mag[scienceFOV_index[count]])
                print("2")
            if i in LGS3_pixels:
                cutoutstars_index.append(i)
                scienceFOV_stars.remove(i)
                cutoutstars_LGS.append(3)
                print(self.mag[scienceFOV_index[count]])
                print("3")
            if i in LGS4_pixels:
                cutoutstars_index.append(i)
                scienceFOV_stars.remove(i)
                cutoutstars_LGS.append(4) 
                print(self.mag[scienceFOV_index[count]])
                print("4")
        
        print("Stars in FOV = "+ str(len(techFOV_stars)))
        print("Stars in annulus = "+str(len(techFOVonly_stars)))
        print("Stars in science field = "+str(len(scienceFOV_stars)))
        print("Stars obscured by LGS cutouts = "+str(len(cutoutstars_index)))
        print("LGS cutout obscured stars lie in = "+str(cutoutstars_LGS))
        for i in footprint_index:
            print(self.mag[i])

        if report == True:

            print("")
            period = time.time() - start #End time of findstars
            print("For NSIDE/Resolution of 2^{}/{:.2} arcsec".format((np.log2(self.NSIDE)),hp.nside2resol(self.NSIDE, arcmin=True) * 60))
            print("Time taken = {:.2}s".format(period))
            #print("============================================")            
            return period,hp.nside2resol(self.NSIDE, arcmin=True) * 60

In [72]:
a=FOV_stars(NSIDE=2**18)
a.loadstars()

Time to convert stars to pixels = 0.15s


In [73]:
a.findstars(53.16571426,-27.89375114, LGS_angle=20, report=True)

tech
16.3867244720459
2
science
16.156139373779297
2
16.213136672973633
2
17.005924224853516
3
18.14518928527832
4
18.255395889282227
1
18.29361915588379
3
18.34016990661621
2
18.856996536254883
2
Stars in FOV = 23
Stars in annulus = 7
Stars in science field = 7
Stars obscured by LGS cutouts = 9
LGS cutout obscured stars lie in = [2, 2, 2, 3, 4, 1, 3, 2, 2]
16.3867244720459
18.255395889282227

For NSIDE/Resolution of 2^18.0/0.81 arcsec
Time taken = 1.4s


(1.3727631568908691, 0.8051921277697045)

In [70]:
a.findstars(54.1,-27.8, LGS_angle=10, report=True)

tech
13.926642417907715
3
14.266542434692383
3
science
12.709394454956055
3
13.70879077911377
1
14.418876647949219
4
Stars in FOV = 14
Stars in annulus = 4
Stars in science field = 5
Stars obscured by LGS cutouts = 5
LGS cutout obscured stars lie in = [3, 3, 3, 1, 4]
13.926642417907715
14.266542434692383
13.70879077911377

For NSIDE/Resolution of 2^18.0/0.81 arcsec
Time taken = 0.6s


(0.6005377769470215, 0.8051921277697045)

In [ ]:
a=FOV_stars(NSIDE=2**15)
a.loadstars()
a.findstars(53.14884949,-27.91123886, report=True)

In [ ]:
times=[]
resolutions=[]
for i in range(10,22):
    a=FOV_stars(NSIDE=2**i)
    a.loadstars()
    t,R=a.findstars(54.022,-27.8, report=True)
    times.append(t)
    resolutions.append(R)
    
    
    
    

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(resolutions,times,marker="x")
plt.xlabel("Resolutions (arcsec)")
plt.ylabel("Time to Retrieve Stars (s)")
#plt.ylim(0,0.4)
#plt.xlim(0,60)
plt.yscale('log')
plt.xscale('log')

In [ ]:
for i in range(0,len(times)):
    #print("t: "+str(times[i])+" Res: "+str(resolutions[i]))
    print("t: {:.2}s, Res: {:.2} arcsecs".format(times[i],resolutions[i]))

In [ ]:
a=FOV_stars(NSIDE=2**17)
a.loadstars()

In [ ]:
a.findstars(53.10850525,-27.71668243, report=True)

In [ ]:
# count=1
# pixelcount=[]
# for i in range(0,len(techFOV_pixels)-1):
#     if techFOV_pixels[i+1]==techFOV_pixels[i]+1:
#         count=count+1
#     else:
#         pixelcount.append(count)
#         count=1
# pixelcount.append(count)


In [ ]:
# print(pixelcount)

In [ ]:
# print(len(pixelcount))